In [ ]:
"""
2 - Unsupervised Learning: Clustering
In this part your task is to query Ethereum transactions and use unsupervised learning in order to cluster addresses. 
Using all transactions from the last 1-2 weeks should be enough to tackle this task.

The task is deliberately held general. Here are a couple of things to consider:

1. Which features do you engineer in order to feed into the clustering algorithm?
2. Can you tell which features had the most influence on your clustering results? Do you see interesting patterns, e.g. anything in particular that is common for addresses in a particular cluster?
3. How do you chose the inital number of clusters?
4. Is it possible to evaluate the results of your clustering? If so, how?
5. How would you tackle this task if you had to run feature engineering and clustering on a year of transaction data, 
e.g. the data would be too large to fit into the memory of your computer and computation of features would be computationally very expensive?
6. What are more advanced things you can think of on how to tackle this task, but that are out of this challenge's scope?
"""

from google.cloud import bigquery
import pandas as pd
import numpy as np

BLOCK_TIMESTAMP_FROM = "'2019-04-20 00:00:00'" 
#TODO: replace BLOCK_TIMESTAMP_TO with the current date
BLOCK_TIMESTAMP_TO = "'2019-04-27 00:00:00'" 

client = bigquery.Client()

sql = """
SELECT *
    FROM `bigquery-public-data.ethereum_blockchain.transactions` as t
    WHERE t.block_timestamp >= {from_block_ts}
    AND t.block_timestamp <= {to_block_ts}
""".format(from_block_ts=BLOCK_TIMESTAMP_FROM, to_block_ts=BLOCK_TIMESTAMP_TO)

df_token_tran = client.query(sql).to_dataframe()
df_token_tran.head()

# https://scikit-learn.org/stable/modules/clustering.html
# https://scikit-learn.org/stable/tutorial/statistical_inference/unsupervised_learning.html

In [ ]:
"""
1 - Blockchain Data Insights
Here are some questions we would like you to solve you in this task:

Which ERC20 tokens had the most activity (in terms of number of transactions) within the last week?
What is the total transaction volume of each of those top tokens within that period?
How does the daily number of transactions of those top 5 tokens look like for the last month?
Think about how to best represent (visualise) the results obtained from the above questions.
"""

from google.cloud import bigquery
import pandas as pd
import numpy as np


# TODO: replace BLOCK_TIMESTAMP with the 7 days before the current date
# the following instructions cause error
# Forbidden: 403 GET https://dp.kaggle.net/bigquery/v2/projects/kaggle-161607/jobs/a0c4ed59-067c-4d34-93cd-33d79e3d47ae?location=US: Forbidden: [bigquery-public-data.ethereum_blockchain]
date_N_days_ago = datetime.now() - timedelta(days=7)
date_N_days_ago = date_N_days_ago.strftime("%Y-%m-%d %H:%M:%S")
date_N_days_ago = "'{}'".format(date_N_days_ago)
BLOCK_TIMESTAMP = date_N_days_ago

BLOCK_TIMESTAMP_FROM = "'2019-04-20 00:00:00'" 
#TODO: replace BLOCK_TIMESTAMP_TO with the current date
BLOCK_TIMESTAMP_TO = "'2019-04-27 00:00:00'" 

client = bigquery.Client()

# SUM(CAST(tx.value AS float64)/POWER(10,18)) as amount_sent,
sql = """
SELECT tx.token_address, COUNT(tx.transaction_hash) as nunm_transactions,
    SUM(CAST(tx.value AS float64)) as total_volume_transacction,
    MIN(block_timestamp) as start_block_timestamp,
    {to_block_ts} as current_timestamp
    FROM `bigquery-public-data.ethereum_blockchain.token_transfers` as tx
    WHERE tx.block_timestamp >= {from_block_ts}
    AND tx.block_timestamp <= {to_block_ts}
    GROUP BY 1
    ORDER BY nunm_transactions DESC
    LIMIT 5
""".format(from_block_ts=BLOCK_TIMESTAMP_FROM, to_block_ts=BLOCK_TIMESTAMP_TO)

df_token_tran = client.query(sql).to_dataframe()

# 1. Which ERC20 tokens had the most activity (in terms of number of transactions) within the last week?
print('The ERC20 tokens had the most activity (in terms of number of transactions) within the last week')
print(df_token_tran.iloc[0]['token_address'])
print(df_token_tran.iloc[0]['nunm_transactions'])

# 2. What is the total transaction volume of each of those top tokens within that period?
print('The total transaction volume of each of those top tokens within that period')
print(df_token_tran['total_volume_transacction'])

# 3. How does the daily number of transactions of those top 5 tokens look like for the last month?
def get_daily_num_transaction(BLOCK_TIMESTAMP_FROM, BLOCK_TIMESTAMP_TO, token_address):
    sql = """
    SELECT *
    FROM `bigquery-public-data.ethereum_blockchain.token_transfers` as tx
    WHERE tx.block_timestamp >= {from_block_ts}
    AND tx.block_timestamp <= {to_block_ts}
    AND tx.token_address = '{token_address}'
    ORDER BY block_timestamp DESC
    """.format(from_block_ts=BLOCK_TIMESTAMP_FROM, to_block_ts=BLOCK_TIMESTAMP_TO, token_address=token_address)

    df_token_tran = client.query(sql).to_dataframe()
    df_token_tran_new = df_token_tran[['block_timestamp']]
    df_token_tran_new['block_timestamp'] = pd.to_datetime(df_token_tran['block_timestamp'])
    
    df_token_tran_new['num_transaction'] = 1
    df_token_tran_new.index = df_token_tran_new['block_timestamp'] 

    #https://chrisalbon.com/python/data_wrangling/pandas_group_data_by_time/
    #df_token_tran_new.resample('D').sum()
    df_token_tran_new = df_token_tran_new.resample('D').sum()
    return df_token_tran_new

#TODO: replace BLOCK_TIMESTAMP_TO with the current date and BLOCK_TIMESTAMP_FROM with one month before
BLOCK_TIMESTAMP_FROM = "'2019-03-27 00:00:00'" 
BLOCK_TIMESTAMP_TO = "'2019-04-27 00:00:00'" 

#token_address = df_token_tran.iloc[0]['token_address']
#df_token_tran_daily = get_daily_num_transaction(BLOCK_TIMESTAMP_FROM, BLOCK_TIMESTAMP_TO, token_address)
#print('number of daily transactions')
#df_token_tran_daily

for token_address in df_token_tran['token_address'].tolist():
    df_token_tran_daily = get_daily_num_transaction(BLOCK_TIMESTAMP_FROM, BLOCK_TIMESTAMP_TO, token_address)
    print('token address: {}'.format(token_address))
    print('number of daily transactions')
    print(df_token_tran_daily)    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


from google.cloud import bigquery
import pandas as pd
